# analysis fastReach 

# 1. general prep


In [ ]:

# load packages
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from statannotations.Annotator import Annotator
# %load_ext rpy2.ipython # currently not needed
import glob

from wordcloud import WordCloud, STOPWORDS
#from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import pymer4 as pymer4

#for anova
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.anova import AnovaRM
from statannot import add_stat_annotation


In [ ]:
# set repository
# p= "C://Users/terfu/Desktop/2021-fastReach/data/study/" #repository for output
p = '/Volumes/Lukas_Gehrke/fastReach/data/'

# 2. temporal binding



## prep
### load data

In [ ]:
# load data
behavior_raw= pd.read_csv( p + 'PI_results_design.csv')



# set data types
behavior_raw[['ed','rt']] = behavior_raw[['ed','rt']].apply(pd.to_numeric, axis=1)

behavior_raw['rd'] = behavior_raw['rd'].astype("string")  # needs to be string before category for R conversion

behavior_raw[['rd','condition']] = behavior_raw[['rd','condition']].astype("category")


# delete VP 18 (did not follow instructions in block 3)

behavior_raw =  behavior_raw.loc[behavior_raw["id"] !=18]



### check raw data (rection times and estimated times)


In [ ]:
fig, ax = plt.subplots()
hist = sns.histplot(behavior_raw, x = 'rt',ax =ax, hue = 'id')
#ax.set_xlim(0,5) # remove outliers for visualization

fig, ax = plt.subplots()
hist = sns.histplot(behavior_raw, x = 'ed',ax =ax, hue = 'id')


fig, ax = plt.subplots()
hist = sns.histplot(behavior_raw, x = 'delta_tap_ems',ax =ax, hue = 'id')
ax.set_xlim(0,5) # remove outliers for visualization

### clean data

In [ ]:
# removes outliers trials that have outliers either in rt or ed
#@Lukas: do we want to remove only extreme or all outliers?

cols = ['rt', 'ed','delta_tap_ems'] # relevant cols

# calculate quantiles and IQR
Q1 = behavior_raw[cols].quantile(0.25) # Same as np.percentile but maps (0,1) and not (0,100)
Q3 = behavior_raw[cols].quantile(0.75)
IQR = Q3 - Q1

# return a boolean array of the rows with (any) non-outlier column values
condition = ~((behavior_raw[cols] < (Q1 - 3 * IQR)) | (behavior_raw[cols] > (Q3 + 3 * IQR))).any(axis=1)

# filter our dataframe based on condition
behavior = behavior_raw[condition]
behavior_del = behavior_raw[-condition]

In [ ]:
# check outlier removel

#count deleted trials
print('Deleted trials:',len(behavior_del))

#per pID
print('Deleted trials per pID')
print(behavior_del['id'].value_counts())

print('mean')
print(behavior_del['id'].value_counts().mean())
print('sd')
print(behavior_del['id'].value_counts().std())

#per condition
print('Deleted trials per condition')
print(behavior_del['condition'].value_counts())
print('mean')
print(behavior_del['condition'].value_counts().mean())
print('sd')
print(behavior_del['condition'].value_counts().std())

print(len(behavior))


In [ ]:
#check new data

fig, ax = plt.subplots()
hist = sns.histplot(behavior, x = 'rt',ax =ax, hue = 'id')
#ax.set_xlim(0,5) # remove outliers for visualization

fig, ax = plt.subplots()
hist = sns.histplot(behavior, x = 'ed',ax =ax, hue = 'id')

fig, ax = plt.subplots()
hist = sns.histplot(behavior, x = 'delta_tap_ems',ax =ax, hue = 'id')


## plot
### prep plot

In [ ]:
# with stimulation condition + time condition+ rd as y

behavior[['rd']] = behavior[['rd']].apply(pd.to_numeric, axis=1)
behavior[['ed']] = behavior[['ed']].apply(pd.to_numeric, axis=1)
# diff = ed - 350 because 350 is the mean of the different time intervals; maybe add *-1  to adapt direction of the plot
behavior["diff"]= behavior['ed']
behavior[['rd']] = behavior[['rd']].astype("category")


####tbd - change diff to rd-ed to get positive value for understimation
#behavior["diff"] = behavior["diff"]*-1

behavior_means = behavior.groupby(['id','condition'],as_index=False)['diff'].mean()

palette = ['#576683', '#E4f392', '#337775']

cats= ['baseline','ems_random','ems_bci'] 
ylabel = 'ed -350'
xlabel = 'condition'
title = 'temporal_binding'
data = behavior
data_means = behavior_means
y = 'diff'
x = "condition"
hue = "id"
hue2 = 'rd'



### plot


@ Lukas: double check; does this make sense to you?

 I used diff (ed-rd) as the depent measure. This basically describes the underestimation of the duration between touch and tone. Larger negative values describe larger underestimation, i.e., interval was 200 ms (rd)  but was estimated 100ms (ed), the dependet measure would be -100 ms. A value of 0 depicts a matching estimate. 

We assume that high agency leads to increased underestimation (temporal binding), therewith the baseline should have the lowest vaule, passive the highest (around 0) and agency should be somewhere inbetween. 

Would it make sense to invert the graphic/diff (*-1) - because high in underestimation is connected to high agency = both graphics would go the same direction.


In [ ]:
# runs plot but needs post processing in affinty desiger
cats_0 = data[data[x]==cats[0]]
cats_1 = data[data[x]==cats[1]]
cats_2 = data[data[x]==cats[2]]

pairs = [(cats[0], cats[1],cats[2])]


with sns.plotting_context('paper', font_scale = 1.8):

    ### Create new plot
    fig, ax = plt.subplots(1, 1, figsize=(4,4))
    fig.patch.set_alpha(1)

    sns.despine() #bottom=True, left=True

        # show boxplots
    ax = sns.boxplot(data = data,x = x, y = y,  order= ['baseline','ems_random','ems_bci'] ,  palette= palette )
    for patch in ax.patches: # adapt alpha
            r, g, b, a = patch.get_facecolor()
            patch.set_facecolor((r, g, b, .9))

    # show line connecting means
    ax =  sns.pointplot(data = data,x = x, y = y, markers="s" ,color = 'black', order = cats)
    
     #  show lines connecting pid means observations    
    ax = sns.lineplot(data = data_means, x = x, y = y,hue = hue, palette = sns.color_palette(['black'],8),legend = False,alpha=0.4)
    
    ax = sns.scatterplot(data=data_means, x=x, y=y,s=10, legend= True, marker="s",color = 'black',edgecolor = 'black',alpha=0.4)
        
    # ax.invert_yaxis()
    # add_stat_annotation(ax, data=data, x=x, y=y, box_pairs=[("baseline", "ems_random"), ("baseline", "ems_bci"), ('ems_random', 'ems_bci')], test='t-test_paired', text_format='star', loc='outside', verbose=2)

    handles, labels = ax.get_legend_handles_labels()  
    
    #plt.legend(handles[3:6],labels[3:6],frameon=True,loc = 'upper left',labelspacing =0.3)
    
    # label_plot_for_subcats(ax)
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    
    plt.show()

fig.savefig('results/'+title + '_'+'.eps', format='eps', transparent=True, bbox_inches='tight', dpi=300)
fig.savefig('results/'+title + '_'+'.svg', format='svg', transparent=True, bbox_inches='tight', dpi=300)
fig.savefig('results/'+title + '_' +'.png', format='png', transparent=True, bbox_inches='tight', dpi=300)

## anaysis


### descriptive

In [ ]:
# descreptive
print(behavior.groupby('condition')['ed'].agg(['mean','std']))

#print(behavior.groupby('rd')['diff'].agg(['mean','std']))



### LMM

#### prep
- witched from rpy2 to pymer4, because it is less annoying with importing / exporting from python to R
- not sure if of this is still necessary, but since pymer4 is based on rpy2, this might be the case

In [ ]:
# packnames = ('lme4', 'lmerTest', 'emmeans', 'geepack', 'sjPlot')
# from rpy2.robjects.packages import importr
# from rpy2.robjects.vectors import StrVector
# utils = importr("utils")
# utils.chooseCRANmirror(ind=1)

In [ ]:
# packnames = ('lme4', 'lmerTest', 'emmeans', 'geepack', 'sjPlot', 'Matrix')
# from rpy2.robjects.vectors import StrVector
# utils.install_packages(StrVector(packnames))


#### built models

In [ ]:
from pymer4.models import Lmer

In [ ]:
# full model

full_model = Lmer("diff ~ condition + (1|id)", data = behavior)
display(full_model.fit(REML = False, factors={"condition": ["baseline", "ems_random", "ems_bci"]}))
#full_model.plot_summary()


#### define null models

In [ ]:

# null model without stimulation condition 
null_model = Lmer("diff ~ (1|id)", data = behavior)
null_model.fit(REML = False, summarize=False)


#### liklyhood ratio tests



stimulation condition

In [ ]:
pymer4.stats.lrt([null_model,full_model])

temp condition

#### post hoc test

TODO check wether this post_hoc test is correct


 @Lukas: Hier bin ich mir nicht sicher, ob man d as einfach so machen kann. Bzw. Also pairwise t-test mit dem full model. 
- in dem 2019 CPS Paper wird die Quelle verwendet für post-hoc test
 check - Wilcoxon, F.: Individual comparisons by ranking methods. Biom. Bull. 1(6), 80
(1945) 
- der code kommt von hier:  http://eshinjolly.com/pymer4/auto_examples/example_03_posthoc.html



In [ ]:
# post hoc test condition
marginal_estimates, comparisons = full_model.post_hoc(
    marginal_vars="condition",p_adjust = "bonf")

print(comparisons)


# 3. subjective data



In [ ]:
questionaire= pd.read_excel(p + 'questionaire.xlsx') 

good_pid = ['12', '14', '15', '16', '17', '19', '20', '21']
questionaire['pID'] = questionaire['pID'].astype(str)
questionaire = questionaire.loc[questionaire['pID'].isin(good_pid)]


questionaire_long = questionaire.melt(id_vars= ['pID'],value_vars=['baseline','passive','agency'])

### plot control

In [ ]:
palette = ['#576683', '#E4f392', '#337775']
cats= ['item_baseline', 'item_passive', 'item_agency']
ylabel = 'level of control'
xlabel = 'condition'
title = 'subjective control rating'
data = questionaire_long
y = 'value'
x = 'variable'
hue = 'pID'

#cohend(data,x,y, cats)

In [ ]:
#adapt

with sns.plotting_context('paper', font_scale = 1.8):

        ### Create new plot
        fig, ax = plt.subplots(1, 1, figsize=(4,4))
        fig.patch.set_alpha(1)

        sns.despine() #bottom=True, left=True
         # show boxplots
        ax = sns.boxplot(data = data, x = x, y = y,  palette= palette)
        for patch in ax.patches: # adapt alpha
             r, g, b, a = patch.get_facecolor()
             patch.set_facecolor((r, g, b, .9))

                   
        # show line connecting means
        sns.pointplot(
            data = data,x = x, y = y, markers="s", color = 'black')
        
        # pId means
        
        ax = sns.lineplot(data = data, x = x, y = y,hue = hue, palette = sns.color_palette(['black'],8),legend = False,alpha=0.4)
    
        ax = sns.scatterplot(data=data, x=x, y=y,s=10, legend= True, marker="s",color = 'black',edgecolor = 'black',alpha=0.4)

        #sns.scatterplot(data = data,x = x, y = y, markers="^",color = 'black')
        add_stat_annotation(ax, data=data, x=x, y=y, box_pairs=[("baseline", "passive"), ("baseline", "agency"), ('passive', 'agency')], test='t-test_ind', text_format='star', loc='outside', verbose=2)

        
        # Label and show
        # label_plot_for_subcats(ax)
        ax.set_title(title)
        ax.set_ylabel(ylabel)
        ax.set_xlabel(xlabel)

      

        plt.show()
        fig.savefig('results/'+title + '_' +'.svg', format='svg', transparent=True, bbox_inches='tight', dpi=300)
        fig.savefig('results/'+title + '_' +'.png', format='png', transparent=True, bbox_inches='tight', dpi=300)
        fig.savefig('results/'+title + '_'+'.eps', format='eps', transparent=True, bbox_inches='tight', dpi=300)

#### LME


In [ ]:
#full model
full_model = Lmer('value ~ variable + (1|pID)', data = questionaire_long)
display(full_model.fit(REML = False,factors={"variable": ["baseline", "passive", "agency"]}))


In [ ]:
#null model
null_model = Lmer('value ~ (1|pID)', data = questionaire_long)
null_model.fit(REML = False, summarize=False)

In [ ]:
# liklyhood ratio test
pymer4.stats.lrt([null_model,full_model])

In [ ]:

# post hoc test condition
marginal_estimates, comparisons = full_model.post_hoc(
    marginal_vars="variable",p_adjust = "bonf")

print(comparisons)

In [ ]:
# participants
print(questionaire.age.mean())
print(questionaire.age.std())

# Plot EEG

In [ ]:

slope= pd.read_csv( p + 'PI_results_design_slope_processed.csv')
#data cleaning happend somewhere else




In [ ]:
palette = ['#576683', '#337775']
cats= ['idle', 'pre_move']
ylabel = 'slope'
xlabel = 'condition'
title = 'slope'
data = slope
y = 'slopes'
x = 'condition'
hue = 'id'

In [ ]:
# plot a box plot of the slope values by condition
#slope.boxplot(column=['slopes'], by=['condition'], figsize=(12,8))


with sns.plotting_context('paper', font_scale = 1.8):

        ### Create new plot
        fig, ax = plt.subplots(1, 1, figsize=(3,4))
        fig.patch.set_alpha(1)

        sns.despine() #bottom=True, left=True
         # show boxplots
        ax = sns.boxplot(data = data, x = x, y = y,  palette= palette)
        for patch in ax.patches: # adapt alpha
             r, g, b, a = patch.get_facecolor()
             patch.set_facecolor((r, g, b, .9))

                   
        # show line connecting means
        sns.pointplot(data = data,x = x, y = y, markers="s" ,color = 'black')
        
        # pId means
        
        ax = sns.lineplot(data = data, x = x, y = y,hue = hue, palette = sns.color_palette(['black'],8),legend = False,alpha=0.4)
    
        ax = sns.scatterplot(data=data, x=x, y=y,s=10, legend= True, marker="s",color = 'black',edgecolor = 'black',alpha=0.4)

        #sns.scatterplot(data = data,x = x, y = y, markers="^",color = 'black')

        plt.gca().invert_yaxis()

        # add statannotation for the pair 'idle' and 'pre_move'
        add_stat_annotation(ax, data=data, x=x, y=y, box_pairs=[("idle", "pre_move")], test='t-test_ind', text_format='star', loc='outside', verbose=2)
        

        # Label and show
        # label_plot_for_subcats(ax)
        # ax.set_title(title)
        ax.set_ylabel(ylabel)
        ax.set_xlabel(xlabel)

      

        plt.show()
        fig.savefig('results/'+title + '_' +'.svg', format='svg', transparent=True, bbox_inches='tight', dpi=300)
        fig.savefig('results/'+title + '_' +'.eps', format='eps', transparent=True, bbox_inches='tight', dpi=300)
        fig.savefig('results/'+title + '_' +'.png', format='png', transparent=True, bbox_inches='tight', dpi=300)





In [ ]:
stats.ttest_ind(slope['slopes'][slope['condition'] == 'idle'], slope['slopes'][slope['condition'] == 'pre_move'])

# 3. Content Analysis
--> not yet adapted to new file format
- built to world clouds for each block
- define stopwords
- maybe check sentiment analysis -> pareid t-test?
-TODO: try word cloud german


In [ ]:
interview_baseline = questionaire['text_baseline'].dropna().to_json(force_ascii = False)

interview_passive = questionaire['text_passive'].dropna().to_json(force_ascii = False)

interview_agency = questionaire['text_agency'].dropna().to_json(force_ascii = False)

stopwords = STOPWORDS # extend by domaine specific words if necessary

interview_baseline2 = questionaire['text_baseline'].dropna().to_string(force_ascii = False)

In [ ]:
#todo check the difference to this code

interview_baseline2 = questionaire['text_baseline'].dropna().to_string()

In [ ]:
print(interview_baseline2)

In [ ]:
wc_formate = WordCloud(background_color='black', max_words=500, width=3000,
                    height=1500, stopwords=stopwords, min_font_size=2,
                    contour_width=3, contour_color='white')

wc_formate.generate(interview_baseline2)
wc_formate.to_file(("results/wordcloud_baseline.png"))

wc_formate.generate(interview_passive)
wc_formate.to_file(("results/wordcloud_passive.png"))

wc_formate.generate(interview_passive)
wc_formate.to_file(("results/wordcloud_agency.png"))


# 4. Sentiment Analysis

In [ ]:

analyzer=SentimentIntensityAnalyzer()  

print(analyzer.polarity_scores(interview_baseline))
print(analyzer.polarity_scores(interview_passive))
print(analyzer.polarity_scores(interview_agency))


